For more information:
* SQL: http://www.sqlcourse.com/intro.html
* sqlite3: https://docs.python.org/2/library/sqlite3.html
* https://medium.com/analytics-vidhya/programming-with-databases-in-python-using-sqlite-4cecbef51ab9
* About relationships: http://www.sqlitetutorial.net/sqlite-foreign-key/
* More about relationships: https://www.sqlite.org/foreignkeys.html
* SQLAlchemy: https://www.sqlalchemy.org

### DB

A database is an organized collection of data, generally stored and accessed electronically from a computer system

### SQL (pronounced "ess-que-el")

1. stands for Structured Query Language
2. it is the standard language for relational database management systems
3. Standard commands: "Select", "Insert", "Update", "Delete", "Create", and "Drop"

### sqlite

0. C library that provides a lightweight disk-based database
1. doesn’t require a separate server process
2. helpful to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL

module sqlite3 provides an interface for sqlite.

It it useful to use an app to see a created DB (for example DB Browser, https://sqlitebrowser.org)

In [ ]:
import sqlite3

In [ ]:
# connect to db
# ":memory:" allows to create db in RAM. Useful for tests
conn = sqlite3.connect(':memory:')  # another param: 'example.db' - save on disk

In [ ]:
# A cursor is just an abstraction. Remember to work through it
# https://stackoverflow.com/questions/6318126/why-do-you-need-to-create-a-cursor-when-querying-a-sqlite-database
c = conn.cursor()

# Create table
c.execute('''
    CREATE TABLE stocks
    (date text, trans text, symbol text, qty real, price real)
''')
# in execute argument should be **SQL commands** to perform

# Insert a row of data
c.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
conn.commit()

In [ ]:
# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

Some tutorials on SQL commands:
1. https://beginner-sql-tutorial.com/sql-commands.htm
2. https://www.codecademy.com/articles/sql-commands

In [ ]:
# Simple example
conn = sqlite3.connect(':memory:')
c = conn.cursor()
# PRIMARY KEY is unique and allows fast indexes and resolves collisions
c.execute('''
    CREATE TABLE test (
        id INTEGER PRIMARY KEY,
        name TEXT,
        age INTEGER
)''')

c.execute("INSERT INTO test(name, age) VALUES ('John', 27)")


for row in c.execute("SELECT id, name FROM test"):
    print(row)

## Another way:
# c.execute("SELECT id, name FROM test").fetchall()

## Update a value in the table:
# c.execute("UPDATE test set name = 'Paul' WHERE ID = 1")
# next(c.execute("SELECT id, name FROM test"))

## Delete a value
# c.execute("DELETE from test WHERE ID = 1")
# list(c.execute("SELECT id, name FROM test"))

In [ ]:
## You can even read a table with pandas
# player_table = pd.read_sql_query("SELECT * FROM test", con)

#### You can't sharing connections and cursors between threads

### Task 1

* Create database workers.db
* Create table Employee with following attributes:

| Column      | Type    |
|-------------|---------|
| employee_id | integer |
| name        | string  |
| months      | integer |
| salary      | integer |

* Fill DB with values:

| employee_id | name     | months | salary |
|-------------|----------|--------|--------|
| 12228       | Rose     | 15     | 1968   |
| 33645       | Angela   | 1      | 3443   |
| 45692       | Frank    | 17     | 1608   |
| 56118       | Patrick  | 7      | 1345   |
| 59725       | Lisa     | 11     | 2330   |
| 74197       | Kimberly | 16     | 4372   |
| 78454       | Bonnie   | 8      | 1771   |
| 83565       | Michael  | 6      | 2017   |
| 98607       | Todd     | 5      | 3396   |
| 99989       | Joe      | 9      | 3573   |

* Check corectness with DB Browser or an alternative
* Write a query that prints a list of employee names (1)
* Write a query that prints a list of employee names who have maximum salary (2)
* Write a query that prints a number of employees who have maximum total earnings. Earnings = month * salary (3)

You will have to use:

1. COUNT
2. GROUP BY
3. ORDER BY
4. LIMIT

In [ ]:
# Your code goes here

Use values from Python variables in SQL operations (https://xkcd.com/327/):

Use **?** as a placeholder

In [ ]:
# Insert many records at a time
conn = sqlite3.connect(':memory:')
c = conn.cursor()

c.execute('''
    CREATE TABLE stocks (
        date text,
        trans text,
        symbol text, 
        qty real,
        price real
)''')

purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
            ]
c.executemany('INSERT INTO stocks VALUES (?,?,?,?,?)', purchases)

### Relationships

Types of relationships between tables:

* One To Many
* One To One
* Many To Many

**Primary** and **foreign** keys define the relational structure of a database. These keys enable each row in the database tables to be identified and define the relationships between the tables.

1. The **primary key** of a relational table uniquely identifies each record in the table. It is a column, or set of columns, that allows each row in the table to be uniquely identified. No two rows in a table with a primary key can have the same primary key value.
2. A **foreign key** is a field in a relational table that matches the primary key column of another table.


<img src="https://cdn-images-1.medium.com/max/2600/1*yW_ha3z8Mp6fUn9m6qWwNw.png" width="600">


### One to many relationship

```
CREATE TABLE IF NOT EXISTS suppliers (
 supplier_id integer PRIMARY KEY,
 supplier_name text NOT NULL,
 group_id integer NOT NULL
);
 
CREATE TABLE IF NOT EXISTS supplier_groups (
 group_id integer PRIMARY KEY,
 group_name text NOT NULL
);
```

#### Drawbacks of this approach:

1. there is no mechanism to prevent you from adding a row to the suppliers table without a corresponding row in the supplier_groups table
2. you may remove a row in the supplier_groups table without deleting or updating the corresponding rows in the suppliers table. This may leave orphaned rows in the suppliers table

** To enforce this relationship, you use a foreign key constraint**:
```
DROP TABLE IF EXISTS suppliers;
 
CREATE TABLE suppliers (
 supplier_id INTEGER PRIMARY KEY,
 supplier_name TEXT NOT NULL,
 group_id INTEGER NOT NULL,
        FOREIGN KEY (group_id) REFERENCES supplier_groups(group_id)
);
```

* The group_id in the supplier_groups table is called the parent key, which is a column or a set of columns in the parent table that the foreign key constraint refers to
* The group_id column in the suppliers table is called the child key

#### Insertion:

```
INSERT INTO supplier_groups (group_name)
VALUES
 ('Domestic'),('Global'),('One-Time');
```

```
INSERT INTO suppliers (supplier_name, group_id)
VALUES
 ('HP', 2);
```

#### Failed insertion:

```
INSERT INTO suppliers (supplier_name, group_id)
VALUES
 ('ABC Inc.', 4);
```

### Task 2

* Create tables with relationships (Artist, Track):

#### Artist

| Column      | Type    |
|-------------|---------|
| artistid    | integer |
| artistname  | string  |

#### Track

| Column      | Type    |
|-------------|---------|
| trackid     | integer |
| trackname   | string  |
| trackartist | integer |

* Create corresponding classes: Artist, Track
* Code a random generation of artists, tracks
* Save generated artist, tracks into table. Check with DB Browser
* Write a query to get all artists (1)
* Write a query to get all tracks (2)
* Write a query to get all tracks for a specific artist (3)
* Write a query to get an artist for a specific track (4)
* Delete a track for a specific artist, show changes (5)

In [ ]:
# Your code goes here

### SQLAlchemy. The Python SQL Toolkit and Object Relational Mapper (ORM)

SQLAlchemy hides SQL requests under comfy api and treats rows as object. Read an example of usage.

In [ ]:
import sys

#for creating the mapper code
from sqlalchemy import Column, ForeignKey, Integer, String

#for configuration and class code
from sqlalchemy.ext.declarative import declarative_base

#for creating foreign key relationship between the tables
from sqlalchemy.orm import relationship

#for configuration
from sqlalchemy import create_engine

#create declarative_base instance
Base = declarative_base()

# All Classes are here
class Child(Base):
    __tablename__ = 'child'
    id = Column(Integer, primary_key=True)
    name = Column(String(50))
    parent_id = Column(Integer, ForeignKey('parent.id'))
    parent = relationship("Parent", back_populates="children")

    def __repr__(self):
        return "<Child(id={}, name={}, parent_id={})>".format(self.id, self.name, self.parent_id)

class Parent(Base):
    __tablename__ = 'parent'
    id = Column(Integer, primary_key=True)
    name = Column(String(50))  # 50 doesn't matter for sqlite, but it's better to set
    children = relationship("Child", back_populates='parent')
    
    def __repr__(self):
        return "<Parent(id={}, name={})>".format(self.id, self.name)


# creates a create_engine instance at the bottom of the file
# 'echo' flag for logging
engine = create_engine('sqlite:///:memory:', echo=True)  # the same sense as in sqlite3

Base.metadata.create_all(engine)

In [ ]:
from sqlalchemy.orm import sessionmaker

# create a session
Session = sessionmaker(bind=engine)
session = Session()

# add parent
parent1 = Parent(name='Parent_1')
session.add(parent1)  # the instance is pending. parent1 is not in DB

# query parent
qparent1 = session.query(Parent).filter_by(name='Parent_1').first()  # .all(), ets

print('qparent1 is parent1', qparent1 is parent1)

session.add_all([
    Parent(name='Wendy'),
    Parent(name='Mary')])

print('Show new objects:')
print(session.new)

# Also, it is possible to add relations
parent1.children = [Child(name='John'), Child(name='Jessica')]

# fix changes
session.commit()


qparent1 = session.query(Parent).filter_by(name='Parent_1').one()
print(qparent1)
print(qparent1.children)

### Task 3

* Write Artist/Track with SQLAlchemy
* You have to write down a class **Music**, which can operate with a table and objects
* Should be implemented:
  1. Adding an artist (add_artist)
  2. Adding a track to an artist by a name (add_track)
  3. Removing an artist with all his tracks (delete_artist)
  4. Removing a track (delete_track)
  5. Getter for an artist/track
  6. Moving a track from one artist to another by name (move_track)
  7. | After all the points above, add the column 'tracktext' in the Track table ('move data from old db')
  8. Adding track text
  9. Changing track text

In [ ]:
# Your code goes here